# Imporing 

In [3]:
import gym
import random

In [3]:
!pip install gym[atari]

ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'C:\\Users\\mzz30\\anaconda3\\Lib\\site-packages\\~le_py\\zlib1.dll'
Consider using the `--user` option or check the permissions.




  Attempting uninstall: ale-py
    Found existing installation: ale-py 0.8.1
    Uninstalling ale-py-0.8.1:
      Successfully uninstalled ale-py-0.8.1


In [12]:
!pip install gym[atari,accept-rom-license]==0.25.2

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446679 sha256=48ac300f873ab9345f7c46014bb2f6e9f306a44fac4b63fce286efc8aa93f4b6
  Stored in directory: c:\users\mzz30\appdata\local\pip\cache\wheels\b1\1f\f7\2da07cf4f81ea264bdaf043028749d88fe0c2227134a22cf80
Successfully built AutoROM.accept-rom-license


# Trying random actions

In [2]:
env = gym.make("ALE/SpaceInvaders-v5", render_mode = "human")

C:\Users\mzz30\anaconda3\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
C:\Users\mzz30\anaconda3\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
states = env.observation_space.shape
actions = env.action_space.n
states, actions

((210, 160, 3), 6)

In [4]:
episodes = 10
for episode in range(1, episodes + 1):
    stat = env.reset()
    done = False
    score = 0
    while not done:
        action = random.choice([0,1,2,3,4,5])
        back = env.step(action)
        score += back[1]
        done = back[2]
    print(f"Episode: {episode}, Score: {score}")
env.close()

C:\Users\mzz30\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


Episode: 1, Score: 295.0
Episode: 2, Score: 60.0
Episode: 3, Score: 125.0
Episode: 4, Score: 75.0
Episode: 5, Score: 120.0
Episode: 6, Score: 30.0
Episode: 7, Score: 110.0
Episode: 8, Score: 135.0
Episode: 9, Score: 595.0
Episode: 10, Score: 50.0


# Building a DQL Agent

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [14]:
env = gym.make("ALE/SpaceInvaders-v5")
states = env.observation_space.shape
actions = env.action_space.n
states, actions

((210, 160, 3), 6)

In [24]:
model = Sequential([
    Flatten(input_shape= (1, states[0],states[1], states[2])),
    Dense(24, activation = "relu"),
    Dense(24, activation = "relu"),
    Dense(6, activation = "linear")
])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 100800)            0         
                                                                 
 dense_15 (Dense)            (None, 24)                2419224   
                                                                 
 dense_16 (Dense)            (None, 24)                600       
                                                                 
 dense_17 (Dense)            (None, 6)                 150       
                                                                 
Total params: 2,419,974
Trainable params: 2,419,974
Non-trainable params: 0
_________________________________________________________________


In [25]:
Agent = DQNAgent(
    model = model,
    memory = SequentialMemory(limit  = 50000, window_length = 1),
    policy = BoltzmannQPolicy(),
    nb_actions = actions,
    nb_steps_warmup = 10,
    target_model_update = 0.01
)

In [26]:
Agent.compile(Adam(learning_rate = 0.001), metrics= ["mae"])

# Train and Test

In [27]:
Agent.fit(env, nb_steps = 10000, visualize = False, verbose = 1)
results = Agent.test(env, nb_episodes = 10, visualize = True)
env.close()

Training for 10000 steps ...
Interval 1 (0 steps performed)


C:\Users\mzz30\anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    9/10000 [..............................] - ETA: 2:22 - reward: 0.0000e+00

C:\Users\mzz30\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(
C:\Users\mzz30\anaconda3\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
C:\Users\mzz30\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   12/10000 [..............................] - ETA: 32:54 - reward: 0.0000e+00

...

   32/10000 [..............................] - ETA: 1:13:19 - reward: 0.0000e+00

C:\Users\mzz30\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 31 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


 6900/10000 [===================>..........] - ETA: 37:44 - reward: 0.2717done, took 5039.346 seconds
Testing for 10 episodes ...


C:\Users\mzz30\anaconda3\lib\site-packages\gym\core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
C:\Users\mzz30\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:297: UserWarning: WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.
  logger.warn(


Error: render(mode='human') is deprecated. Please supply `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). The new `render_mode` keyword argument supports DPI scaling, audio, and native framerates.

In [29]:
results = Agent.test(env, nb_episodes = 10, visualize = False)
env.close()

Testing for 10 episodes ...
Episode 1: reward: 0.000, steps: 694
Episode 2: reward: 0.000, steps: 694
Episode 3: reward: 0.000, steps: 694
Episode 4: reward: 0.000, steps: 694
Episode 5: reward: 0.000, steps: 694
Episode 6: reward: 0.000, steps: 694
Episode 7: reward: 0.000, steps: 694
Episode 8: reward: 0.000, steps: 694
Episode 9: reward: 0.000, steps: 694
Episode 10: reward: 0.000, steps: 694
